In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision.models as models
# from torch.nn.utils.rnn import pack_padded_sequence
from model import ImageCNN,MatchCNN

import argparse
import os
import pickle
from data_loader import get_loader,CocoDataset
from build_vocab import Vocabulary
from torchvision import transforms
import time
from matchCNN_st import MatchCNN_st

In [7]:
"""parameters"""
image_vector_size = 256
embed_size = 100
margin = 0.1
batch_size = 100
epochs = 1
vocab_size = 9956
momentum=0.9
lr = 0.0001
pad_len = 62
num_workers = 2
batch_size = 100

In [4]:
"""set model"""
imageCNN = ImageCNN(image_vector_size=image_vector_size)
matchCNN = MatchCNN_st(embed_size = embed_size, 
                    image_vector_size = image_vector_size, 
                    vocab_size = vocab_size, 
                    pad_len = pad_len)

if torch.cuda.is_available():
    print("cuda is available")
    imageCNN = imageCNN.cuda()
    matchCNN = matchCNN.cuda()


cuda is available


In [5]:
# """load models"""
# model_path = "../models"
# imageCNN.load_state_dict(torch.load(os.path.join(model_path, 'imageCNN_st0-6072062505563756359988543488.000000.pkl')))
# matchCNN.load_state_dict(torch.load(os.path.join(model_path, 'matchCNN_st0-6072062505563756359988543488.000000.pkl')))

In [6]:
"""set optimizer"""
# params = list(imageCNN.parameters()) + list(matchCNN.parameters())
# params = list(imageCNN.linear.parameters()) + list(imageCNN.bn.parameters()) + list(matchCNN.parameters())
params = list(imageCNN.parameters()) + list(matchCNN.parameters())

optimizer = optim.SGD(params, momentum, lr)



In [7]:
# Load vocabulary wrapper.
with open("../data/coco/vocab.pkl", 'rb') as f:
    vocab = pickle.load(f)

 # Image preprocessing
# For normalization, see https://github.com/pytorch/vision#models
transform = transforms.Compose([ 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

In [8]:
"""load train data"""
# Build data loader
data_loader = get_loader(root = "../data/coco/resized2014", 
                         json = "../data/coco/annotations/captions_train2014.json", 
                         vocab = vocab, 
                         transform = transform, 
                         batch_size = batch_size,
                         shuffle=True,
                         num_workers = num_workers,
                         pad_len = pad_len) 


loading annotations into memory...
Done (t=0.58s)
creating index...
index created!


In [9]:
mean_losses = []
# cudnn.benchmark = True


In [2]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(EncoderCNN, self).__init__()
        resnet = models.resnet18(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
        self.init_weights()
        
    def init_weights(self):
        """Initialize the weights."""
        self.linear.weight.data.normal_(0.0, 0.02)
        self.linear.bias.data.fill_(0)
        
    def forward(self, images):
        """Extract the image feature vectors."""
        features = self.resnet(images)
        features = Variable(features.data)
        features = features.view(features.size(0), -1)
        features = self.bn(self.linear(features))
        return features

NameError: name 'nn' is not defined

In [11]:

encodercnn = EncoderCNN(100)
# encodercnn.cuda()
images = torch.randn(batch_size, 3, 224, 224)
images = Variable(images, volatile = True)
encodercnn.zero_grad()
image_vectors = encodercnn(images)

RuntimeError: cublas runtime error : library not initialized at /pytorch/torch/lib/THC/THCGeneral.c:405

In [ ]:
# for i,(images, images_wrong, captions, lengths) in enumerate(data_loader):
#     print(images)
encodercnn = Enco
images = torch.randn(batch_size, 3, 224, 224)
images = Variable(images, volatile = True).cuda()
imageCNN.zero_grad()
image_vectors = imageCNN(images)
#     break

In [ ]:
start = time.time()
losses = []
# imageCNN.train()
# matchCNN.train()
for epoch in range(1):
#     losses = []
    for i, (images, images_wrong, captions, lengths) in enumerate(data_loader):
        """input data"""
#         image = Variable(torch.randn(batch_size,3,224,224))
#         images_wrong = Variable(torch.randn(batch_size,3,224,224))
#         sentences = Variable(torch.LongTensor(np.random.randint(low=0, high=999, size=(batch_size,pad_len))))
#         captions = 
        if images.size(0) != batch_size:
            break
        
        images = Variable(images, volatile = True)
        images_wrong = Variable(images_wrong, volatile = True)
        captions = Variable(captions)
        if torch.cuda.is_available():
            images = images.cuda()
            images_wrong = images_wrong.cuda()
            captions = captions.cuda()

        imageCNN.zero_grad()
        matchCNN.zero_grad()
        
        """extract imgae feature and embed sentence"""
        image_vectors = imageCNN(images)
        image_vectors_wrong = imageCNN(images_wrong)
#         print(image_vectors)
#         if torch.cuda.is_available():
#             image_vectors_wrong = image_vectors[(torch.randperm(batch_size)).cuda()]
#         else:
#             image_vectors_wrong = image_vectors[torch.randperm(batch_size)]


        """get correct score"""
        scores = matchCNN(image_vectors, captions)
        scores_wrong = matchCNN(image_vectors_wrong, captions)
        target = Variable(torch.ones(batch_size, 1)).cuda()


        """get loss"""
        mrl = torch.nn.MarginRankingLoss(margin=0.1)
        loss = mrl(scores, scores_wrong, target)
        loss.backward()


#         """find nan"""
#         flag = False
#         for parameter in matchCNN.parameters():
#             data_np = parameter.grad.data.cpu().numpy()
#             if np.isnan(data_np).any():
#                 print("nan grad:",parameter.grad)
#                 print("score:", scores[0])
#                 print("i:%s,loss:%s"%(i,loss))
#                 flag = True
#         if flag:
#             break
            
        """update"""     
        optimizer.step()
        losses.append(loss)

        if i % 100 == 0:
            print("-"*30)
            print("last loss:", loss[len(loss)-1])
            print("score:", scores[0])
            print("i:%s,loss:%s"%(i,loss))
                
#         if i == 100:
#             break
#             print("scores",scores[0])
    mean_loss = torch.mean(torch.cat((losses)))
    mean_losses.append(mean_loss)
    print("mean loss:", mean_loss)
    model_path = "../models"
    """save models"""
    torch.save(imageCNN.state_dict(), os.path.join(model_path, 'imageCNN_st%d-%f.pkl'%(epoch,mean_loss.cpu().data.numpy())))
    torch.save(matchCNN.state_dict(), os.path.join(model_path, 'matchCNN_st%d-%f.pkl'%(epoch, mean_loss.cpu().data.numpy())))

print("time used:", time.time() - start)

In [ ]:
print(mean_losses)

In [ ]:
model_path = "../models"
"""save models"""
torch.save(imageCNN.state_dict(), os.path.join(model_path, 'imageCNN.pkl'))
torch.save(matchCNN.state_dict(), os.path.join(model_path, 'matchCNN.pkl'))

In [ ]:
for parameter in matchCNN.parameters():
    print(parameter.grad)

In [ ]:
print(torch.max((scores_wrong - scores + margin,0),dim=1))


In [ ]:
tmp_grad = matchCNN.conv1_sen.weight.grad.data.cpu().numpy()
tmp_grad

In [ ]:
scores = Variable(torch.ones(10,1))
scores_wrong = Variable(torch.ones(10,1))
target = Variable(torch.ones(10,1))

In [ ]:
lossFunc = torch.nn.MarginRankingLoss(margin = 0.5)
loss = lossFunc(scores, scores_wrong, target)
print("loss:",loss)

In [ ]:
# Build val data loader
data_loader_val = get_loader(root = "../data/coco/resized2014", 
                         json = "../data/coco/annotations/captions_val2014.json", 
                         vocab = vocab, 
                         transform = transform, 
                         batch_size = batch_size,
                         shuffle=True,
                         num_workers = num_workers,
                         pad_len = pad_len) 


In [ ]:
mean_loss_val = []

In [ ]:
start = time.time()
target = Variable(torch.ones(batch_size, 1)).cuda()
losses = []
imageCNN.eval()
matchCNN.eval()
for epoch in range(1):
#     losses = []
    for i, (images, captions, lengths) in enumerate(data_loader_val):
        """input data"""
#         image = Variable(torch.randn(batch_size,3,224,224))
#         sentences = Variable(torch.LongTensor(np.random.randint(low=0, high=999, size=(batch_size,pad_len))))
        if images.size(0) != batch_size:
            break
        
        if torch.cuda.is_available():
            images = images.cuda()
            captions = captions.cuda()
        images = Variable(images, volatile = True)
        captions = Variable(captions, volatile = True)
        imageCNN.zero_grad()
        matchCNN.zero_grad()
        
        """extract imgae feature and embed sentence"""
#         imageCNN = imageCNN.cuda()
        image_vectors = imageCNN(images)
#         print(image_vectors)
        if torch.cuda.is_available():
            image_vectors_wrong = image_vectors[(torch.randperm(batch_size)).cuda()]
        else:
            image_vectors_wrong = image_vectors[torch.randperm(batch_size)]


#         """get correct score"""
        scores = matchCNN(image_vectors, captions)
        scores_wrong = matchCNN(image_vectors_wrong, captions)
        print("scores",scores)
        print("scores_wrong",scores_wrong)


#         break
        lossFunc = torch.nn.MarginRankingLoss(margin = 0.5)
#         loss = torch.max(margin + scores_wrong - scores, 0)
        loss = lossFunc(scores, scores_wrong, target)
        losses.append(loss)

        if i % 1 == 0:
            print("i:%s,loss:%s"%(i,loss))
#             print("time used:", time.time() - start)
        if i ==100:
            print("time used:", time.time() - start)
            break
    mean_loss = torch.mean(torch.cat((losses)))
    print("mean loss:", mean_loss)

print("time used:", time.time() - start)

In [ ]:
print(mean_losses)

In [ ]:
5.7672

In [ ]:
CocoDataset = CocoDataset(root = "../data/coco/resized2014", 
                         json = "../data/coco/annotations/captions_val2014.json", 
                       vocab=vocab,
                       transform=transform,
                       pad_len=pad_len)

In [ ]:
coco = CocoDataset.coco

In [ ]:
print(ids[0])
ann = coco.anns[ids[0]]
print(ann)

In [ ]:
anns = coco.anns
imgs = coco.imgs

In [ ]:
print(len(anns))
print(len(imgs))

In [ ]:
anns[87382]

In [ ]:
import collections
ann_extracted = collections.OrderedDict()

In [ ]:
anns[87382]

In [ ]:
ann_extracted.append(anns[87382])

In [ ]:
ann_extracted[0]["caption_ids"] = [1,2,3]

In [ ]:
ann_extracted

In [ ]:
coco.getAnnIds(imgIds = 441695)

In [ ]:
imgs[441695]

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt


In [ ]:
file_path = "../data/coco/resized2014/" + imgs[441695]["file_name"]
img_data = io.imread(file_path)

In [ ]:
plt.imshow(img_data)

In [ ]:
transform = transforms.Compose([ 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

In [ ]:
scores = np.zeros([10,10])
print(scores)

In [ ]:
temp = np.array([])

In [ ]:
temp.put(1)

In [ ]:
lossFunc = torch.nn.MarginRankingLoss(margin = 0.5)

In [ ]:
a = Variable(torch.Tensor([10,11]))

In [ ]:
b = Variable(torch.Tensor([12,10]))

In [ ]:
target = Variable(torch.Tensor([1,1]))

In [ ]:
print(a)
print(b)
print(target)

In [ ]:
lossFunc(a,b,target)

In [ ]:
torch.randperm(batch_size)

In [ ]:
a = torch.LongTensor([1])

In [ ]:
a.random_(0, 10)

In [ ]:
a[0]